# import def func

In [7]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [8]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
#     import pdb;pdb.set_trace()
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        print(eval(f'lastpTDX_DF_{resample}.shape'))
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [9]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [10]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [11]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-20',
 code
 605499    东鹏饮料
 Name: name, dtype: object)

In [12]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/23/25 15:24:29'

code
002948   2025-06-23 15:00:00
002247   2025-06-23 15:00:00
Name: ticktime, dtype: datetime64[ns]

24.49946266412735

#### 实时成交

In [13]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2703992, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-23 14:59:06,66.67,66.98,66.15,66.9,15037292,66.55
2025-06-23 14:59:57,66.67,66.98,66.15,66.9,15037292,66.34
2025-06-23 15:00:00,66.64,66.98,66.15,66.9,15234692,66.89


1.0

总量:9278.9 亿


ticktime    2025-06-24 00:00:00
close                 66.574319
volume                     0.11
vol                       15.23
amount                    10.14
Name: 603019, dtype: object

'2025-06-23'

#### 读取实时成交个股

In [14]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5420, 5),
         volume  name
 code                
 300468    0.85  四方精创
 300308    0.73  中际旭创
 300773    0.68   拉卡拉
 002104    0.47  恒宝股份
 300502    0.46   新易盛
 300059    0.45  东方财富
 600519    0.41  贵州茅台
 688981    0.41  中芯国际
 003040    0.37   楚天龙
 002463    0.37  沪电股份
 601318    0.36  中国平安
 300476    0.35  胜宏科技
 002594    0.34   比亚迪
 002475    0.34  立讯精密
 002074    0.33  国轩高科
 300750    0.33  宁德时代
 300130    0.33   新国都
 300394    0.33  天孚通信
 300180    0.33  华峰超纤
 002366    0.32  融发核电)

##### 合并实时成交量及SinaDM

In [15]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [16]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [17]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [18]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [19]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5181 it:0.7 lastpTDX_DF_3d not in locals
keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-06-23 15:25:01,593] ERROR:tdx_hdf5_api.py(load_hdf_db:729): dratio_limit fn:tdx_last_df cl:5185 h5:2199 don't find:2986 dra:0.58 log_err:0


('TDD:5185',)


  0%|                                                  | 0/5185 [00:00<?, ?it/s]
  0%|                                                                                       | 0.00/5.18k [00:00<?, ?%/s]
  0%|                                                                              | 1.00/5.18k [00:03<4:49:43, 3.35s/%]
  1%|▌                                                                               | 33.0/5.18k [00:05<11:36, 7.40%/s]
  1%|▉                                                                               | 61.0/5.18k [00:07<08:37, 9.90%/s]
  2%|█▍                                                                              | 91.0/5.18k [00:09<07:18, 11.6%/s]
  2%|█▉                                                                               | 121/5.18k [00:11<06:42, 12.6%/s]
  3%|██▎                                                                              | 152/5.18k [00:13<06:12, 13.5%/s]
  4%|██▊                                                                

 39%|██████████████████████████████▌                                                | 2.00k/5.18k [02:24<03:03, 17.3%/s]
 39%|███████████████████████████████                                                | 2.04k/5.18k [02:26<03:01, 17.3%/s]
 40%|███████████████████████████████▌                                               | 2.07k/5.18k [02:28<03:01, 17.1%/s]
 41%|████████████████████████████████▏                                              | 2.11k/5.18k [02:30<03:02, 16.9%/s]
 41%|████████████████████████████████▋                                              | 2.14k/5.18k [02:32<03:01, 16.7%/s]
 42%|█████████████████████████████████▏                                             | 2.18k/5.18k [02:34<02:58, 16.9%/s]
 43%|█████████████████████████████████▋                                             | 2.21k/5.18k [02:36<03:03, 16.2%/s]
 43%|██████████████████████████████████▏                                            | 2.25k/5.18k [02:38<03:03, 16.0%/s]
 44%|███████████████████████████

limit_output extension: Maximum message size of 10000 exceeded with 10002 characters

In [76]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:775 224


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301176,逸豪新材,17.20,5,2025-04-08,28.61,17.86,16.02,1.20,27.98,13.57,25.3,0.00
301219,腾远钴业,14.48,0,2025-04-07,54.56,50.00,43.56,-1.10,54.91,40.97,-0.9,-0.10
300468,四方精创,13.06,30,2025-04-01,44.66,38.62,17.31,0.00,40.28,15.20,0.1,32.31
300773,拉卡拉,12.40,42,2025-04-07,31.72,31.70,18.69,0.00,30.28,13.75,-0.9,20.59
300157,新锦动力,11.95,172,2025-04-07,6.09,5.92,3.55,-0.90,6.12,2.42,1.2,12.70
688230,芯导科技,11.54,0,2025-04-08,54.31,52.19,34.56,-1.10,52.75,35.06,1.3,0.00
600844,丹化科技,10.09,2,2025-04-29,3.60,3.36,2.59,-1.10,3.59,2.78,0.1,1.20
600798,宁波海运,10.09,1,2025-04-08,4.80,4.53,2.59,0.00,4.80,2.71,0.0,0.10
600113,浙江东日,10.05,3,2025-04-08,22.77,14.50,12.42,53.69,22.58,12.58,0.1,-0.10


In [77]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

In [78]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [79]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5176 it:0.7 lastpTDX_DF_w in locals


[2025-06-23 13:25:07,122] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:2716 code:['002037', '301021']
  0%|                                                  | 0/2716 [00:00<?, ?it/s]
  0%|                                                                                       | 0.00/2.72k [00:00<?, ?%/s]
  0%|                                                                              | 1.00/2.72k [00:06<5:02:08, 6.68s/%]
  1%|▌                                                                               | 20.0/2.72k [00:08<15:55, 2.82%/s]
  1%|█▏                                                                              | 39.0/2.72k [00:11<09:45, 4.57%/s]
  2%|█▌                                                                              | 52.0/2.72k [00:13<08:52, 5.00%/s]
  2%|█▉                                                                              | 65.0/2.72k [00:15<08:27, 5.23%/s]
  3%|██▎                                                                    

limit_output extension: Maximum message size of 10000 exceeded with 10120 characters

In [80]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:196 223


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301421,波长光电,19.94,10,2025-04-07,74.81,62.31,52.12,25.00,73.75,43.03,0.0,0.0
600798,宁波海运,10.09,12,2025-01-10,4.80,4.48,2.59,1.20,4.80,2.79,-1.0,1.2
002125,湘潭电化,10.01,121,2025-04-08,13.52,10.37,9.59,42.41,13.52,8.85,0.0,-0.1
600113,浙江东日,10.00,28,2025-01-10,22.77,14.87,11.64,68.50,22.58,12.06,0.0,0.1
603948,建业股份,9.99,4,2025-04-18,21.56,18.59,14.20,29.80,21.56,15.90,0.0,0.1
002235,安妮股份,9.97,557,2025-04-08,9.93,7.33,5.17,38.00,9.59,4.33,22.3,10.5
603928,兴业股份,9.97,71,2025-01-03,15.11,14.44,7.38,37.00,15.07,7.74,14.1,-1.1
003040,楚天龙,6.86,253,2025-04-08,24.94,19.20,12.37,36.09,23.70,10.71,5.3,1.1
688638,誉辰智能,6.84,0,2025-01-10,35.59,32.41,24.70,1.20,34.03,26.28,1.3,1.2


In [81]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

macd          0.24
macddif       0.34
macddea       0.22
macdlast2    -0.12
macdlast3    -0.09
lasth1d      17.27
lastl1d       14.4
lastl2d      14.22
lastl3d      13.92
lastp1d      17.27
ma51d        15.03
ma52d        14.57
ma53d        14.76
ma54d        14.82
Name: 301038, dtype: object

### resample M

In [82]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx 

[2025-06-23 13:30:26,107] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:2716 code:['002037', '301021']


:0 b1>:5176 it:0.6 lastpTDX_DF_m in locals


  0%|                                                  | 0/2716 [00:00<?, ?it/s]
  0%|                                                                                       | 0.00/2.72k [00:00<?, ?%/s]
  0%|                                                                              | 1.00/2.72k [00:04<3:05:52, 4.11s/%]
  1%|▌                                                                               | 19.0/2.72k [00:06<12:04, 3.72%/s]
  1%|█                                                                               | 38.0/2.72k [00:08<07:42, 5.79%/s]
  2%|█▋                                                                              | 57.0/2.72k [00:10<06:28, 6.85%/s]
  3%|██▏                                                                             | 76.0/2.72k [00:12<05:49, 7.56%/s]
  3%|██▊                                                                             | 94.0/2.72k [00:14<05:29, 7.95%/s]
  4%|███▎                                                               

limit_output extension: Maximum message size of 10000 exceeded with 10120 characters

In [83]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:788 224


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
300801,泰和科技,20.02,44,2025-04-07,21.64,17.06,15.52,1.1,20.69,13.86,0.00,0.10
301176,逸豪新材,18.80,45,2025-04-08,29.00,17.86,16.02,1.2,28.02,13.57,25.30,0.00
688321,微芯生物,14.45,16,2025-01-27,24.23,23.00,15.15,1.3,22.17,16.52,-1.10,15.30
688230,芯导科技,12.71,0,2024-09-30,54.88,48.84,26.45,1.2,52.84,26.45,-1.00,35.00
600798,宁波海运,10.09,12,2025-04-30,4.80,3.19,2.49,1.1,4.80,2.59,-1.10,0.10
600844,丹化科技,10.09,139,2025-01-27,3.60,3.11,2.22,1.2,3.59,2.31,1.20,1.20
600794,保税科技,10.08,634,2025-03-31,5.24,3.80,2.81,1.2,5.14,3.36,27.30,0.20
600698,湖南天雁,10.06,123,2024-12-31,7.11,6.40,3.16,-0.9,7.05,4.50,-1.10,1.20
601008,连云港,10.06,285,2025-02-28,7.22,3.97,3.16,-0.9,7.19,3.58,50.91,-0.90


In [84]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d    17.27
lastl1d     14.4
lastl2d    14.22
lastl3d    13.92
lastp1d    17.27
ma51d      15.03
ma52d      14.57
ma53d      14.76
ma54d      14.82
Name: 301038, dtype: object

### resample D

In [85]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx 

[2025-06-23 13:36:01,624] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:2716 code:['002037', '301021']


:0 b1>:5176 it:0.5 lastpTDX_DF_d in locals


  0%|                                                  | 0/2716 [00:00<?, ?it/s]
  0%|                                                                                       | 0.00/2.72k [00:00<?, ?%/s]
  0%|                                                                              | 1.00/2.72k [00:04<3:30:37, 4.65s/%]
  1%|▍                                                                               | 16.0/2.72k [00:06<15:41, 2.87%/s]
  1%|▊                                                                               | 28.0/2.72k [00:09<11:47, 3.80%/s]
  2%|█▎                                                                              | 46.0/2.72k [00:11<08:18, 5.36%/s]
  2%|█▉                                                                              | 66.0/2.72k [00:13<06:33, 6.74%/s]
  3%|██▍                                                                             | 81.0/2.72k [00:15<06:34, 6.68%/s]
  4%|██▊                                                                

limit_output extension: Maximum message size of 10000 exceeded with 10120 characters

In [86]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
# if len(td_code) < 30:
#     write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:23 223


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301421,波长光电,20.00,10,2025-04-07,74.81,62.31,52.12,25.00,73.81,43.03,0.0,0.00
600798,宁波海运,10.09,12,2025-04-08,4.80,4.93,2.96,14.10,4.80,2.71,-0.9,0.00
601606,长城军工,10.02,158,2025-04-07,20.20,14.18,12.67,36.91,20.12,10.79,22.3,1.20
603155,新亚强,10.01,29,2025-04-08,15.83,13.98,12.92,1.30,15.73,11.38,-0.1,0.00
603948,建业股份,9.99,4,2025-04-08,21.56,19.02,17.66,0.10,21.56,14.81,-0.1,-0.10
603022,新通联,9.97,3,2025-04-08,10.26,9.34,8.37,1.30,10.26,6.95,0.0,0.10
603209,兴通股份,9.80,28,2025-04-08,18.05,15.30,14.17,35.69,18.06,13.46,1.2,-0.10
300130,新国都,5.19,116,2025-04-07,30.62,28.77,20.89,0.00,30.09,16.62,-0.9,1.30
688503,聚和材料,4.84,2,2025-04-07,38.59,37.66,30.56,0.20,38.16,29.41,-0.1,37.00


all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/065.blk:23
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx/T0002/blocknew/065.blk:23
write:23 block_path:/Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/065.blk


In [87]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

#### filter test

In [88]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=5)

Count:89 224


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600798,宁波海运,10.09,1,2025-04-08,4.80,4.53,2.59,0.00,4.80,2.71,0.0,0.1
002235,安妮股份,9.97,143,2025-04-08,9.93,7.33,5.17,38.00,9.58,4.33,22.3,10.5
603948,建业股份,9.94,4,2025-04-08,21.56,18.70,14.20,29.09,21.56,14.81,-0.1,1.3
603373,安邦护卫,6.64,0,2025-04-08,45.62,38.69,25.62,0.10,45.53,25.91,14.4,0.1
301238,瑞泰新材,5.28,2,2025-04-07,18.95,18.20,16.70,1.30,18.67,14.33,0.0,0.0


In [89]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [90]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:788
tw_code:196
t3d_code:775
td_code:23
Count:5 223


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600798,宁波海运,10.09,12,2025-04-08,4.80,4.93,2.96,14.1,4.80,2.71,-0.9,0.0
603948,建业股份,9.99,4,2025-04-08,21.56,19.02,17.66,0.1,21.56,14.81,-0.1,-0.1
605580,恒盛能源,4.34,5,2025-04-09,12.50,11.55,10.10,1.2,12.44,8.63,0.0,-0.1
688115,思林杰,3.64,0,2025-04-07,48.97,45.22,37.72,1.3,47.81,29.59,0.0,5.3
601916,浙商银行,2.58,375,2025-04-07,3.58,3.44,3.02,1.2,3.54,2.78,0.1,0.1


#### check data

In [91]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

5

In [92]:
show_tdx_data(td_code.loc[code_f])

Count:5 223


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600798,宁波海运,10.09,12,2025-04-08,4.80,4.93,2.96,14.1,4.80,2.71,-0.9,0.0
603948,建业股份,9.99,4,2025-04-08,21.56,19.02,17.66,0.1,21.56,14.81,-0.1,-0.1
605580,恒盛能源,4.34,5,2025-04-09,12.50,11.55,10.10,1.2,12.44,8.63,0.0,-0.1
688115,思林杰,3.64,0,2025-04-07,48.97,45.22,37.72,1.3,47.81,29.59,0.0,5.3
601916,浙商银行,2.58,375,2025-04-07,3.58,3.44,3.02,1.2,3.54,2.78,0.1,0.1


### Write toblk

In [93]:
# write_code_to_blk(code_f,'077',rewrite=False)